In [151]:
import numpy as np
from PIL import Image
import glob

IMG_SHAPE = (250, 250)


def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    print(flat_array)

    return flat_array


def vector_to_img(vector, shape = IMG_SHAPE):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img


def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [166]:
all_imgs = []

# Get images from dataset and convert them to vectors
for folder in glob.iglob('../dataset/*'):
    for img in glob.iglob(folder + '/*'):
        all_imgs.append(img_to_vector(img))

# Create an np.array from the vectors
training_set = np.array(all_imgs)


In [167]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [178]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
sub = None
for v in training_set:
    sub = np.subtract(v, avg_face)
    A.append(sub)

print(np.array(A).shape)

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = np.array(A).transpose()

(225, 62500)


In [179]:
# Form the covariance matrix
cov_matrix = np.cov(np.array(A))

In [180]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
eigvectors_sort

[array([ 0.06731064,  0.00611241, -0.0254702 , -0.11892324,  0.07692986,
        -0.04946996, -0.04437823, -0.00735828,  0.09380905, -0.00630101,
         0.00723245,  0.03360128, -0.0796121 , -0.04168188, -0.0357486 ,
         0.08846413,  0.00874869,  0.03096761, -0.00330366,  0.10778488,
        -0.0728557 , -0.0179904 ,  0.04043429,  0.00767106, -0.01077956,
        -0.17088042, -0.04783995, -0.10541565,  0.0956199 , -0.01326326,
        -0.15094299, -0.01107212, -0.09283851,  0.07475781, -0.02570743,
         0.10001833, -0.02383532,  0.03585948, -0.03849146,  0.08207836,
        -0.00047093,  0.05568885,  0.02941   , -0.04324604, -0.11588193,
        -0.02257639,  0.0451054 , -0.00675643, -0.17450714, -0.02448619,
         0.08491764,  0.03136223, -0.07464139, -0.09126929, -0.01890673,
        -0.09194177,  0.00426539, -0.04631297, -0.05441821, -0.00363935,
         0.06634424, -0.1771447 ,  0.01428426,  0.04163078, -0.04638788,
        -0.09854672, -0.05443498,  0.09076339,  0.0

In [181]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:7]).transpose()

In [182]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()
# Calculate weights for eigenfaces
w = np.array([np.dot(proj_data,i) for i in np.array(A)])

In [208]:
# calculate eigenface of image in question
test_image = img_to_vector('../dataset/j_lopez/Jennifer_Lopez_0010.jpg')
mean_test_image = np.subtract(test_image, avg_face)

In [209]:
print(test_image)
w_unknown = np.dot(proj_data, test_image)
w_unknown

[0 0 0 ... 0 0 0]


array([-2.61703910e+09, -8.20120569e+08, -3.95630478e+08,  5.15225619e+08,
       -5.53547936e+07,  4.08481038e+08,  2.83051702e+06])

In [210]:
diff  = w - w_unknown
print(diff.shape)
norms = np.linalg.norm(diff, axis=1)
print(norms.shape)
min(norms)
index = np.argmin(norms)
index

(225, 7)
(225,)


213

In [211]:
vector_to_img(training_set[index]).show()